<a href="https://colab.research.google.com/github/Bravinkindi9/Car_Brand_Recognition/blob/main/Urban_DNA_Sequencer_Phase1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
import geemap
import pandas as pd


In [2]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KDTree
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import folium
from folium import plugins
import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
try:
    # Try to connect immediately
    ee.Initialize(project='rwanda-urban-dna') #(id)
except Exception as e:
    # If that fails, ask for permission, then connect
    ee.Authenticate()
    # MAKE SURE THIS ID MATCHES EXACTLY
    ee.Initialize(project='rwanda-urban-dna')        #(id number)

In [4]:
# STEP 2: DEFINE OUR LAB (KIGALI)
# We focus on the city center first to test quickly
kigali_center = ee.Geometry.Point([30.0619, -1.9441])
roi = kigali_center.buffer(3000) # 3km radius test circle


In [5]:
# STEP 3: LOAD THE 2023 DATA
# This is the V3 dataset you found!
buildings = ee.FeatureCollection('GOOGLE/Research/open-buildings/v3/polygons') \
    .filterBounds(roi) \
    .filter(ee.Filter.gt('confidence', 0.75)) # Only confident buildings!!!!(0.50 is good but its large for nothing, we use 75 it has middle and low and high classified houses)
      #Only give me buildings where the AI is more than 75%

count = buildings.size().getInfo()
print("Count of the buildings", {count})


Count of the buildings {48191}


In [6]:
# STEP 4: VISUALIZE (To prove we have it)
Map = geemap.Map(center=[-1.9441, 30.0619], zoom=14)
Map.addLayer(buildings, {'color': 'red'}, 'Kigali Buildings 2023')
Map

Map(center=[-1.9441, 30.0619], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

**Part 2**: **Feature Engineering**




# STEP 5: CALCULATE BUILDING "DNA" (Geometry)

In [7]:
def add_dna(feature):
    # 1. Get the geometry of the building
    geom = feature.geometry()

    # 2. Calculate the Metrics (The Math)
    area = geom.area()
    perimeter = geom.perimeter()

    # 3. Calculate "Compactness" (The Shape Index)
    # Formula: Perimeter^2 / Area
    # Simple shapes (Squares) have LOW numbers.
    # Complex shapes (Messy Polygons) have HIGH numbers.
    shape_index = perimeter.pow(2).divide(area)

    # 4. Attach these numbers to the building data
    return feature.set({
        'building_area': area,
        'building_perimeter': perimeter,
        'shape_index': shape_index
    })

# Apply this function to every single building in the list
# This runs on the server, so it calculates 50,000 buildings in 1 second.
dna_buildings = buildings.map(add_dna)

In [8]:

# TEST: Look at the first building to prove it worked
print(dna_buildings.first().toDictionary().select(['building_area', 'shape_index']).getInfo())

{'building_area': 144.60665714206036, 'shape_index': 18.46131506818793}


# Spatial Join:
The reason is because we haven't well found the houses! For each building, only look for neighbors within 20 meters. Ignore the rest.

so, we are to find the shape on the reality of the earth terrain instead of relying only on the housing data(area, perimeter, complexity)

In [9]:
# STEP 6: CALCULATE SPATIAL CONTEXT (Nearest Neighbor)

# 1. Define the "Search Radius"
# We assume if a neighbor is further than 20m, it's not "crowded".
max_dist_filter = ee.Filter.withinDistance(
    distance=20,
    leftField='.geo',
    rightField='.geo',
    maxError=1
)

# 2. Define the Join
# "saveBest" means: Find the closest neighbor, save it, and ignore the others.
join = ee.Join.saveBest(
    matchKey='nearest_neighbor',
    measureKey='distance_to_neighbor'
)

# 3. Apply the Join
# We join the dataset to itself! (Building A looks for Building B)
buildings_with_neighbors = join.apply(dna_buildings, dna_buildings, max_dist_filter)

# 4. Clean up the result
# The Join creates a complex structure. We need to flatten it back to a simple number.
def flatten_neighbor(feature):
    neighbor = feature.get('nearest_neighbor')

    # If a neighbor was found, get the distance.
    # If NO neighbor was found (it's lonely), set distance to 99 meters.
    dist = ee.Feature(neighbor).get('distance_to_neighbor')
    final_dist = ee.Algorithms.If(neighbor, dist, 99)

    return feature.set({'nearest_dist': final_dist})

final_data = buildings_with_neighbors.map(flatten_neighbor)

In [10]:
# STEP 1: DOWNLOAD RAW DATA
!pip install geemap -q
import ee
import geemap
import pandas as pd
import numpy as np
from sklearn.neighbors import KDTree # The tool for Step 2

# Connect
try:
    ee.Initialize(project='rwanda-urban-dna')
except:
    ee.Authenticate()
    ee.Initialize(project='rwanda-urban-dna')

# Define Kigali Center (2km radius)
roi = ee.Geometry.Point([30.0619, -1.9441]).buffer(2000)

# Feature Engineering Part A: Geometry (Area & Shape)
# We do this part in GEE because it's fast there.
def add_geometry_features(feature):
    geom = feature.geometry()
    area = geom.area()
    perimeter = geom.perimeter()
    # Shape Index: Perimeter^2 / Area
    shape_index = perimeter.pow(2).divide(area)

    return feature.set({
        'building_area': area,
        'shape_index': shape_index,
        'lat': geom.centroid().coordinates().get(1),
        'lon': geom.centroid().coordinates().get(0)
    })

print("⏳ Downloading Raw Data & Geometry Features...")
buildings = ee.FeatureCollection('GOOGLE/Research/open-buildings/v3/polygons') \
    .filterBounds(roi) \
    .filter(ee.Filter.gt('confidence', 0.75)) \
    .map(add_geometry_features)

# Export to Pandas Table - Use ee_to_df instead
df = geemap.ee_to_df(buildings)
# Select only the columns we need
df = df[['building_area', 'shape_index', 'lat', 'lon']]
print(f"✅ Step 1 Complete: Downloaded {len(df)} buildings.")

# STEP 2: FEATURE ENGINEERING PART B: NEIGHBORS (The "New Way")
print("⏳ Calculating Nearest Neighbors (Python Method)...")

# We build a 'Spatial Index' of all building locations
coordinates = df[['lat', 'lon']].values
tree = KDTree(coordinates, metric='euclidean')

# Find the 2 closest points for every building
# (Point 1 is itself, Point 2 is the neighbor)
distances, indices = tree.query(coordinates, k=2)

# Convert degrees to meters (approx) and save to table
# distances[:, 1] means "take the 2nd closest point"
df['nearest_dist'] = distances[:, 1] * 111139

print("✅ Step 2 Complete: Feature Engineering Finished!")
print(df.head())

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 36.3 MB/s eta 0:00:00
⏳ Downloading Raw Data & Geometry Features...
✅ Step 1 Complete: Downloaded 21035 buildings.
⏳ Calculating Nearest Neighbors (Python Method)...
✅ Step 2 Complete: Feature Engineering Finished!
   building_area  shape_index       lat        lon  nearest_dist
0     223.829924    21.792427 -1.956578  30.074076     19.810851
1      46.195452    15.988332 -1.930959  30.057113      6.380950
2      44.920176    16.005849 -1.949705  30.046144      8.803278
3      27.863373    15.929239 -1.945238  30.044880      6.411638
4      75.518470    21.544221 -1.946042  30.045883     13.621331


- **Row 0** (The Rich House?):Area: 223 $m^2$ (Huge!)Distance: 19.8m (Lots of privacy)Verdict: This looks like a planned villa.

- **Row 1** (The Informal House?):Area: 46 $m^2$ (Tiny)Distance: 6.3m (Packed tight!)Verdict: This looks like a crowded, informal settlement.

# **K-Means Algorithm**

Here, we are to make all the rows(classes) be on the same scale!
- We use a famous ML theory called ***Standard Scaler***

**StandardScaler: Centers the average at 0**

This is to handle Outliers like Factories and big infrastructures without breaking the math for small houses

# Phase 2: Density Analysis & Intelligent Clustering

This script adds local density features and performs unsupervised learning
to identify informal settlements in Kigali building footprints

In [11]:
# STEP 1: CALCULATE LOCAL DENSITY.

def calculate_local_density(df, radius_meters=50):
    """
    Calculate the number of neighboring buildings within a given radius.

    Parameters:
    df : pd.DataFrame
        Must contain 'lat' and 'lon' columns
    radius_meters : float
        Search radius in meters (default: 50m)

    Returns:
    pd.Series
        Count of neighbors within radius for each building
    """

    print("Step 1: Calculating density within", radius_meters, "m radius")

    # Convert meters to degrees (approximate)
    # At equator: 1 degree ≈ 111,139 meters
    radius_degrees = radius_meters / 111139

    # Build KDTree for efficient spatial queries
    coords = df[['lat', 'lon']].values
    tree = KDTree(coords)

    # Query: for each point, count neighbors within radius
    # (subtract 1 to exclude the point itself)
    counts = tree.query_radius(coords, r=radius_degrees, count_only=True) - 1

    print(f" Density calculated. Range: {counts.min()} to {counts.max()} neighbors")
    return counts



In [12]:

# STEP 2: RE-TRAIN K-MEANS WITH NORMALIZED FEATURES

def train_kmeans_with_scaling(df, features, n_clusters=3):
    """
    Train K-Means with proper feature scaling.

    Parameters:
    df : pd.DataFrame
        Building footprint data
    features : list
        Column names to use for clustering
    n_clusters : int
        Number of clusters (default: 3)

    Returns:
    --------
    tuple
        (cluster_labels, scaler, kmeans_model)
    """
    print(f"\n Step 2: Training K-Means (k={n_clusters}) with StandardScaler...")

    # Extract features
    X = df[features].values

    # Normalize features (CRITICAL for mixing area & density)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    print(f" Feature ranges BEFORE scaling:")
    for i, feat in enumerate(features):
        print(f"   - {feat}: [{X[:, i].min():.2f}, {X[:, i].max():.2f}]")

    # Train K-Means
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(X_scaled)

    print(f"   ✅ Clustering complete. Cluster sizes: {np.bincount(cluster_labels)}")
    return cluster_labels, scaler, kmeans



In [13]:
# STEP 3: AUTO-LABEL CLUSTERS (INTELLIGENT MAPPING)

def auto_label_clusters(df, cluster_col='cluster_id'):
    """
    Automatically identify and label clusters based on characteristics.

    Logic:
    - Informal Settlement: Highest density + Smallest area
    - Commercial: Largest area (regardless of density)
    - Planned: Everything else (medium density, medium area)

    Parameters:
    -----------
    df : pd.DataFrame
        Must contain cluster_col, 'building_area', 'density_50m'
    cluster_col : str
        Name of cluster column

    Returns:
    --------
    pd.DataFrame
        DataFrame with new 'settlement_type' column
    """
    print(f"\n🏘️  Step 3: Auto-labeling clusters...")

    # Calculate mean characteristics per cluster
    cluster_profiles = df.groupby(cluster_col).agg({
        'building_area': 'mean',
        'density_50m': 'mean'
    }).round(2)

    print("\n   Raw cluster profiles:")
    print(cluster_profiles)

    # Find cluster IDs for each type
    informal_cluster = cluster_profiles['density_50m'].idxmax()  # Highest density
    commercial_cluster = cluster_profiles['building_area'].idxmax()  # Largest area

    # Planned is the remaining cluster
    all_clusters = set(cluster_profiles.index)
    planned_cluster = list(all_clusters - {informal_cluster, commercial_cluster})[0]

    # Map cluster IDs to meaningful labels
    label_map = {
        informal_cluster: 'Informal',
        commercial_cluster: 'Commercial',
        planned_cluster: 'Planned'
    }

    df['settlement_type'] = df[cluster_col].map(label_map)

    print(f"\n   ✅ Label mapping:")
    print(f"      - Cluster {informal_cluster} → 'Informal' (high density, small area)")
    print(f"      - Cluster {commercial_cluster} → 'Commercial' (large area)")
    print(f"      - Cluster {planned_cluster} → 'Planned' (medium density/area)")

    return df



In [14]:
# STEP 4: OUTPUT VERIFICATION SUMMARY

def print_cluster_summary(df):
    """
    Print detailed summary statistics for verification.
    """
    print("\n" + "="*70)
    print(" FINAL CLUSTER PROFILES (Verification)")
    print("="*70)

    summary = df.groupby('settlement_type').agg({
        'building_area': ['mean', 'std', 'count'],
        'density_50m': ['mean', 'std'],
        'shape_index': 'mean',
        'nearest_dist': 'mean'
    }).round(2)

    print(summary)

    # Sanity check
    print("\n✅ Sanity Check:")
    informal_density = df[df['settlement_type'] == 'Informal']['density_50m'].mean()
    planned_density = df[df['settlement_type'] == 'Planned']['density_50m'].mean()

    if informal_density > planned_density:
        print("   ✓ Informal has HIGHER density than Planned (as expected)")
    else:
        print("    WARNING: Informal density is NOT higher than Planned!")

    informal_area = df[df['settlement_type'] == 'Informal']['building_area'].mean()
    commercial_area = df[df['settlement_type'] == 'Commercial']['building_area'].mean()

    if commercial_area > informal_area:
        print("   ✓ Commercial has LARGER area than Informal (as expected)")
    else:
        print("    WARNING: Commercial area is NOT larger than Informal!")



In [15]:
# MAIN EXECUTION PIPELINE

def main(df):
    """
    Main execution pipeline for density analysis and clustering.

    Parameters:
    -----------
    df : pd.DataFrame
        Must contain columns: ['lat', 'lon', 'building_area', 'shape_index', 'nearest_dist']

    Returns:
    --------
    pd.DataFrame
        Enhanced dataframe with 'density_50m', 'cluster_id', 'settlement_type'
    """
    print("="*70)
    print(" URBAN DNA SEQUENCER - PHASE 2: DENSITY & CLUSTERING")
    print("="*70)
    print(f"Input: {len(df):,} buildings in Kigali\n")

    # Step 1: Add density feature
    df['density_50m'] = calculate_local_density(df, radius_meters=50)

    # Step 2: Train K-Means with scaling
    features = ['building_area', 'shape_index', 'nearest_dist', 'density_50m']
    cluster_labels, scaler, kmeans = train_kmeans_with_scaling(df, features, n_clusters=3)
    df['cluster_id'] = cluster_labels

    # Step 3: Auto-label clusters
    df = auto_label_clusters(df, cluster_col='cluster_id')

    # Step 4: Print summary
    print_cluster_summary(df)

    print("\n" + "="*70)
    print(" PIPELINE COMPLETE")
    print("="*70)
    print(f"New columns added: ['density_50m', 'cluster_id', 'settlement_type']")
    print(f"\nTo use: df_enhanced = main(df)")

    return df


# USAGE EXAMPLE (FOR COLAB)

if __name__ == "__main__":
    # Assuming 'df' is already loaded in your Colab environment
    # Run this script with:
    # %run urban_dna_density_clustering.py
    # df_enhanced = main(df)

    print("\n💡 To run this script in Colab:")
    print("   1. Make sure 'df' is loaded with columns: lat, lon, building_area, shape_index, nearest_dist")
    print("   2. Run: df_enhanced = main(df)")
    print("   3. Inspect: df_enhanced[['settlement_type', 'density_50m', 'building_area']].head()")


💡 To run this script in Colab:
   1. Make sure 'df' is loaded with columns: lat, lon, building_area, shape_index, nearest_dist
   2. Run: df_enhanced = main(df)
   3. Inspect: df_enhanced[['settlement_type', 'density_50m', 'building_area']].head()


In [16]:
df =  main(df)

 URBAN DNA SEQUENCER - PHASE 2: DENSITY & CLUSTERING
Input: 21,035 buildings in Kigali

Step 1: Calculating density within 50 m radius
 Density calculated. Range: 0 to 57 neighbors

 Step 2: Training K-Means (k=3) with StandardScaler...
 Feature ranges BEFORE scaling:
   - building_area: [3.02, 5721.05]
   - shape_index: [15.12, 95.73]
   - nearest_dist: [1.46, 133.15]
   - density_50m: [0.00, 57.00]
   ✅ Clustering complete. Cluster sizes: [ 7970 12030  1035]

🏘️  Step 3: Auto-labeling clusters...

   Raw cluster profiles:
            building_area  density_50m
cluster_id                            
0                  133.83        13.89
1                   77.97        33.36
2                  588.92         7.34

   ✅ Label mapping:
      - Cluster 1 → 'Informal' (high density, small area)
      - Cluster 2 → 'Commercial' (large area)
      - Cluster 0 → 'Planned' (medium density/area)

 FINAL CLUSTER PROFILES (Verification)
                building_area                density_50m  

!!!!! We see in the cluster that we have some target houses of Informal(83m+ 9.7 m close)

And Actually we have more than 21k points on the map! so, we are going to continue

___



# **VISUALIZING THE RESULTS.**

---



# Interactive Map Visualization

Creates an interactive Folium map to visualize building classifications
overlaid on satellite imagery for visual verification.

Generate an interactive map with building classifications on satellite imagery.
- Parameters:
  df : pd.DataFrame
- Must contain columns: ['lat', 'lon', 'settlement_type', 'building_area', 'density_50m']
- output_file : str
- Output HTML filename (default: 'kigali_urban_dna.html')
= Returns:
    folium.map
  

In [17]:
def create_kigali_urban_map(df, output_file='kigali_urban_dna.html'):
    print("="*70)
    print("🗺️  GENERATING INTERACTIVE KIGALI URBAN DNA MAP")
    print("="*70)

In [18]:
# STEP 1: CALCULATE MAP CENTER
center_lat = df['lat'].mean()
center_lon = df['lon'].mean()

print(f"\n Map Center: ({center_lat:.6f}, {center_lon:.6f})")
print(f" Total Buildings: {len(df):,}")

# Count buildings by type
type_counts = df['settlement_type'].value_counts()
print(f"\n  Building Distribution:")
for settlement_type, count in type_counts.items():
    print(f"   - {settlement_type}: {count:,} ({count/len(df)*100:.1f}%)")


 Map Center: (-1.943998, 30.058912)
 Total Buildings: 21,035

  Building Distribution:
   - Informal: 12,030 (57.2%)
   - Planned: 7,970 (37.9%)
   - Commercial: 1,035 (4.9%)


In [19]:
# STEP 2: CREATE BASE MAP WITH SATELLITE IMAGERY
print(f"\n  Creating base map with Esri WorldImagery...")

# Create map centered on Kigali
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=13,
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Esri WorldImagery'
)


  Creating base map with Esri WorldImagery...


In [20]:
# STEP 3: DEFINE COLOR SCHEME
color_map = {
        'Informal': '#FF0000',      # Red
        'Planned': '#0000FF',       # Blue
        'Commercial': '#FFFF00'     # Yellow
    }

print(f"\n Color Scheme:")
for settlement_type, color in color_map.items():
    print(f"   - {settlement_type}: {color}")


 Color Scheme:
   - Informal: #FF0000
   - Planned: #0000FF
   - Commercial: #FFFF00


In [21]:

    # STEP 4: ADD BUILDINGS AS CIRCLE MARKERS
print(f"\n Adding {len(df):,} building markers...")

    # Create feature groups for each settlement type (for layer control)
feature_groups = {
        settlement_type: folium.FeatureGroup(name=f"{settlement_type} ({count:,})")
        for settlement_type, count in type_counts.items()
    }

    # Add markers to respective feature groups
for idx, row in df.iterrows():
  settlement_type = row['settlement_type']

   # Create tooltip with building info
tooltip_text = f"""
<b>Type:</b> {settlement_type}<br>
<b>Area:</b> {row['building_area']:.1f} m²<br>
<b>Density (50m):</b> {row['density_50m']} buildings
"""

        # Add circle marker
folium.CircleMarker(
    location=[row['lat'], row['lon']],
    radius=2,  # Small radius to avoid clutter
    color=color_map[settlement_type],
    fill=True,
    fillColor=color_map[settlement_type],
    fillOpacity=0.7,
    weight=1,
    tooltip=tooltip_text
    ).add_to(feature_groups[settlement_type])

        # Progress indicator for large datasets
if (idx + 1) % 5000 == 0:
  print(f"   ... {idx + 1:,} / {len(df):,} processed")

    # Add all feature groups to map
for fg in feature_groups.values():
        fg.add_to(m)



 Adding 21,035 building markers...


In [22]:
# STEP 5: ADD LAYER CONTROL & EXTRAS
print(f"\n  Adding layer control and fullscreen option...")

# Add layer control (toggle settlement types on/off)
folium.LayerControl(collapsed=False).add_to(m)

# Add fullscreen button
plugins.Fullscreen(
    position='topright',
    title='Fullscreen',
    title_cancel='Exit Fullscreen',
    force_separate_button=True
).add_to(m)

# Add minimap for navigation
minimap = plugins.MiniMap(toggle_display=True)
m.add_child(minimap)



  Adding layer control and fullscreen option...


In [23]:
import os
def create_kigali_urban_map(df, output_file='kigali_urban_dna.html'):
  print(f"\n💾 Saving map to '{output_file}'...")
  m.save(output_file)
  file_size_mb = os.path.getsize(output_file) / (1024 * 1024)
  print("="*70)
  print("✅ MAP GENERATION COMPLETE")
  print("="*70)
  print(f"📄 Output File: {output_file}")
  print(f"📦 File Size: {file_size_mb:.2f} MB")
  print(f"\n🚀 To view: Open '{output_file}' in your browser")
  print(f"💡 Tip: Use Layer Control (top-right) to toggle settlement types")
  print(f"🔍 Tip: Hover over points to see building details")
  print("="*70)
  return m


In [24]:
# EXECUTE THE VISUALIZATION
# 1. Generate the map (The Full Version)
m = create_kigali_urban_map(df, output_file='kigali_urban_dna.html')

# 2. Download the HTML file to your computer
from google.colab import files
files.download('kigali_urban_dna.html')


💾 Saving map to 'kigali_urban_dna.html'...
✅ MAP GENERATION COMPLETE
📄 Output File: kigali_urban_dna.html
📦 File Size: 0.01 MB

🚀 To view: Open 'kigali_urban_dna.html' in your browser
💡 Tip: Use Layer Control (top-right) to toggle settlement types
🔍 Tip: Hover over points to see building details


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [25]:
"""
Urban DNA Sequencer - Interactive Map Visualization
====================================================
Creates an interactive Folium map to visualize building classifications
overlaid on satellite imagery for visual verification.

Author: Claude
Date: 2026-02-10
"""

import folium
from folium import plugins
import pandas as pd
import numpy as np

def create_kigali_urban_map(df, output_file='kigali_urban_dna.html'):
    """
    Generate an interactive map with building classifications on satellite imagery.

    Parameters:
    -----------
    df : pd.DataFrame
        Must contain columns: ['lat', 'lon', 'settlement_type', 'building_area', 'density_50m']
    output_file : str
        Output HTML filename (default: 'kigali_urban_dna.html')

    Returns:
    --------
    folium.Map
        The generated map object
    """

    print("="*70)
    print("🗺️  GENERATING INTERACTIVE KIGALI URBAN DNA MAP")
    print("="*70)

    # =============================================================================
    # STEP 1: CALCULATE MAP CENTER
    # =============================================================================
    center_lat = df['lat'].mean()
    center_lon = df['lon'].mean()

    print(f"\n📍 Map Center: ({center_lat:.6f}, {center_lon:.6f})")
    print(f"📊 Total Buildings: {len(df):,}")

    # Count buildings by type
    type_counts = df['settlement_type'].value_counts()
    print(f"\n🏘️  Building Distribution:")
    for settlement_type, count in type_counts.items():
        print(f"   - {settlement_type}: {count:,} ({count/len(df)*100:.1f}%)")

    # =============================================================================
    # STEP 2: CREATE BASE MAP WITH SATELLITE IMAGERY
    # =============================================================================
    print(f"\n🛰️  Creating base map with Esri WorldImagery...")

    # Create map centered on Kigali
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=13,
        tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr='Esri WorldImagery'
    )

    # =============================================================================
    # STEP 3: DEFINE COLOR SCHEME
    # =============================================================================
    color_map = {
        'Informal': '#FF0000',      # Red
        'Planned': '#0000FF',       # Blue
        'Commercial': '#FFFF00'     # Yellow
    }

    print(f"\n🎨 Color Scheme:")
    for settlement_type, color in color_map.items():
        print(f"   - {settlement_type}: {color}")

    # =============================================================================
    # STEP 4: ADD BUILDINGS AS CIRCLE MARKERS
    # =============================================================================
    print(f"\n📌 Adding {len(df):,} building markers...")

    # Create feature groups for each settlement type (for layer control)
    feature_groups = {
        settlement_type: folium.FeatureGroup(name=f"{settlement_type} ({count:,})")
        for settlement_type, count in type_counts.items()
    }

    # Add markers to respective feature groups
    for idx, row in df.iterrows():
        settlement_type = row['settlement_type']

        # Create tooltip with building info
        tooltip_text = f"""
        <b>Type:</b> {settlement_type}<br>
        <b>Area:</b> {row['building_area']:.1f} m²<br>
        <b>Density (50m):</b> {row['density_50m']} buildings
        """

        # Add circle marker
        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=2,  # Small radius to avoid clutter
            color=color_map[settlement_type],
            fill=True,
            fillColor=color_map[settlement_type],
            fillOpacity=0.7,
            weight=1,
            tooltip=tooltip_text
        ).add_to(feature_groups[settlement_type])

        # Progress indicator for large datasets
        if (idx + 1) % 5000 == 0:
            print(f"   ... {idx + 1:,} / {len(df):,} processed")

    # Add all feature groups to map
    for fg in feature_groups.values():
        fg.add_to(m)

    # =============================================================================
    # STEP 5: ADD LAYER CONTROL & EXTRAS
    # =============================================================================
    print(f"\n🎛️  Adding layer control and fullscreen option...")

    # Add layer control (toggle settlement types on/off)
    folium.LayerControl(collapsed=False).add_to(m)

    # Add fullscreen button
    plugins.Fullscreen(
        position='topright',
        title='Fullscreen',
        title_cancel='Exit Fullscreen',
        force_separate_button=True
    ).add_to(m)

    # Add minimap for navigation
    minimap = plugins.MiniMap(toggle_display=True)
    m.add_child(minimap)

    # =============================================================================
    # STEP 6: SAVE MAP
    # =============================================================================
    print(f"\n💾 Saving map to '{output_file}'...")
    m.save(output_file)

    # Calculate file size
    import os
    file_size_mb = os.path.getsize(output_file) / (1024 * 1024)

    print("="*70)
    print("✅ MAP GENERATION COMPLETE")
    print("="*70)
    print(f"📄 Output File: {output_file}")
    print(f"📦 File Size: {file_size_mb:.2f} MB")
    print(f"\n🚀 To view: Open '{output_file}' in your browser")
    print(f"💡 Tip: Use Layer Control (top-right) to toggle settlement types")
    print(f"🔍 Tip: Hover over points to see building details")
    print("="*70)

    return m


# =============================================================================
# USAGE EXAMPLE (FOR COLAB)
# =============================================================================

if __name__ == "__main__":
    print("\n💡 USAGE IN COLAB:")
    print("="*70)
    print("# Step 1: Ensure df has the required columns")
    print("required_cols = ['lat', 'lon', 'settlement_type', 'building_area', 'density_50m']")
    print("print(df[required_cols].head())")
    print()
    print("# Step 2: Generate the map")
    print("map_object = create_kigali_urban_map(df)")
    print()
    print("# Step 3: Download the HTML file")
    print("from google.colab import files")
    print("files.download('kigali_urban_dna.html')")
    print("="*70)


# =============================================================================
# ALTERNATIVE: FASTER VERSION FOR VERY LARGE DATASETS (>50k buildings)
# =============================================================================

def create_kigali_urban_map_fast(df, output_file='kigali_urban_dna.html', sample_size=10000):
    """
    Optimized version for large datasets using sampling.

    Parameters:
    -----------
    df : pd.DataFrame
        Building footprint data
    output_file : str
        Output HTML filename
    sample_size : int
        Number of buildings to display (random sample)

    Returns:
    --------
    folium.Map
        The generated map object
    """

    print(f"⚡ FAST MODE: Sampling {sample_size:,} buildings from {len(df):,} total")

    # Stratified sampling to preserve proportions
    df_sampled = df.groupby('settlement_type', group_keys=False).apply(
        lambda x: x.sample(min(len(x), sample_size // 3), random_state=42)
    )

    print(f"✅ Sample size: {len(df_sampled):,} buildings")

    # Use the regular function with sampled data
    return create_kigali_urban_map(df_sampled, output_file)


💡 USAGE IN COLAB:
# Step 1: Ensure df has the required columns
required_cols = ['lat', 'lon', 'settlement_type', 'building_area', 'density_50m']
print(df[required_cols].head())

# Step 2: Generate the map
map_object = create_kigali_urban_map(df)

# Step 3: Download the HTML file
from google.colab import files
files.download('kigali_urban_dna.html')


In [26]:
# EXECUTE THE VISUALIZATION
# 1. Generate the map (The Full Version)
m = create_kigali_urban_map(df, output_file='kigali_urban_dna.html')

# 2. Download the HTML file to your computer
from google.colab import files
files.download('kigali_urban_dna.html')

🗺️  GENERATING INTERACTIVE KIGALI URBAN DNA MAP

📍 Map Center: (-1.943998, 30.058912)
📊 Total Buildings: 21,035

🏘️  Building Distribution:
   - Informal: 12,030 (57.2%)
   - Planned: 7,970 (37.9%)
   - Commercial: 1,035 (4.9%)

🛰️  Creating base map with Esri WorldImagery...

🎨 Color Scheme:
   - Informal: #FF0000
   - Planned: #0000FF
   - Commercial: #FFFF00

📌 Adding 21,035 building markers...
   ... 5,000 / 21,035 processed
   ... 10,000 / 21,035 processed
   ... 15,000 / 21,035 processed
   ... 20,000 / 21,035 processed

🎛️  Adding layer control and fullscreen option...

💾 Saving map to 'kigali_urban_dna.html'...
✅ MAP GENERATION COMPLETE
📄 Output File: kigali_urban_dna.html
📦 File Size: 16.80 MB

🚀 To view: Open 'kigali_urban_dna.html' in your browser
💡 Tip: Use Layer Control (top-right) to toggle settlement types
🔍 Tip: Hover over points to see building details


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Next step: Slope-Based Vulnerability Analysis

Uses Google Earth Engine to analyze landslide risk for Kigali buildings
based on terrain slope from NASA SRTM elevation data

In [27]:
# STEP 1: LOAD NASA SRTM ELEVATION DATA & CALCULATE SLOPE

def get_slope_layer(roi_bounds=None):
    """
    Load SRTM elevation data and calculate slope in degrees.

    Parameters:
    -----------
    roi_bounds : list, optional
        Bounding box [min_lon, min_lat, max_lon, max_lat]
        If None, will use Kigali's approximate bounds

    Returns:
    --------
    ee.Image
        Slope image in degrees
    """
    print("="*70)
    print("  STEP 1: LOADING NASA SRTM ELEVATION DATA")
    print("="*70)

    # Load SRTM Digital Elevation Model (30m resolution)
    srtm = ee.Image('USGS/SRTMGL1_003')
    elevation = srtm.select('elevation')

    print(" SRTM DEM loaded (30m resolution)")

    # Calculate slope in degrees
    slope = ee.Terrain.slope(elevation)

    print("✅ Slope calculated (in degrees)")
    print(f"   Range: 0° (flat) to ~90° (vertical cliff)")

    return slope


In [28]:
# STEP 2: CONVERT DATAFRAME TO EARTH ENGINE FEATURE COLLECTION

def df_to_ee_features(df):
    """
    Convert pandas DataFrame to Earth Engine FeatureCollection.

    Parameters:
    -----------
    df : pd.DataFrame
        Must contain 'lat' and 'lon' columns

    Returns:
    --------
    ee.FeatureCollection
        Point features with building metadata
    """
    print("\n" + "="*70)
    print(" STEP 2: CONVERTING DATAFRAME TO EARTH ENGINE FEATURES")
    print("="*70)

    print(f" Processing {len(df):,} buildings...")

    # Create list of features
    features = []
    for idx, row in df.iterrows():
        # Create point geometry
        point = ee.Geometry.Point([row['lon'], row['lat']])

        # Create feature with properties
        feature = ee.Feature(point, {
            'building_id': int(idx),
            'settlement_type': str(row['settlement_type']),
            'building_area': float(row['building_area']),
            'density_50m': int(row['density_50m'])
        })

        features.append(feature)

        # Progress indicator
        if (idx + 1) % 5000 == 0:
            print(f"   ... {idx + 1:,} / {len(df):,} converted")

    # Convert to FeatureCollection
    fc = ee.FeatureCollection(features)

    print(f"✅ FeatureCollection created with {len(df):,} points")

    return fc



In [29]:
# STEP 3: SAMPLE SLOPE VALUES AT BUILDING LOCATIONS

def sample_slope_at_buildings(fc, slope_image):
    """
    Extract slope values at each building location.

    Parameters:
    -----------
    fc : ee.FeatureCollection
        Building point features
    slope_image : ee.Image
        Slope raster in degrees

    Returns:
    --------
    ee.FeatureCollection
        Features with added 'slope' property
    """
    print("\n" + "="*70)
    print("📍 STEP 3: SAMPLING SLOPE AT BUILDING LOCATIONS")
    print("="*70)

    print("🔄 Running reduceRegions (this may take 30-60 seconds)...")

    # Sample slope values at point locations
    # Use a small buffer (1m) to ensure we get a value even at exact points
    sampled = slope_image.reduceRegions(
        collection=fc,
        reducer=ee.Reducer.first(),  # Get the slope value at the point
        scale=30,  # SRTM resolution is 30m
        tileScale=4  # Increase if memory errors occur
    )

    print("✅ Slope values sampled from SRTM")
    print("   Using 30m resolution (SRTM native scale)")

    return sampled



In [30]:
# STEP 4: CONVERT BACK TO PANDAS & ADD RISK CLASSIFICATION

def ee_to_dataframe(fc):
    """
    Convert Earth Engine FeatureCollection back to pandas DataFrame.

    Parameters:
    -----------
    fc : ee.FeatureCollection
        Features with slope data

    Returns:
    --------
    pd.DataFrame
        DataFrame with slope column
    """
    print("\n" + "="*70)
    print("📥 STEP 4: DOWNLOADING RESULTS FROM EARTH ENGINE")
    print("="*70)

    print("🔄 Fetching data from GEE servers...")
    print("   (This may take 30-90 seconds for 21k buildings)")

    # Convert to list and download
    fc_list = fc.getInfo()['features']

    print(f"✅ Downloaded {len(fc_list):,} records")

    # Extract properties into DataFrame
    data = []
    for feature in fc_list:
        props = feature['properties']
        data.append({
            'building_id': props.get('building_id'),
            'settlement_type': props.get('settlement_type'),
            'building_area': props.get('building_area'),
            'density_50m': props.get('density_50m'),
            'slope': props.get('first', None)  # 'first' is the reducer output
        })

    df_result = pd.DataFrame(data)

    # Handle missing values (water bodies, clouds, etc.)
    missing_count = df_result['slope'].isna().sum()
    if missing_count > 0:
        print(f"  Warning: {missing_count} buildings have missing slope data")
        print(f"   Filling with median slope value...")
        df_result['slope'].fillna(df_result['slope'].median(), inplace=True)

    print(f"\n Slope Statistics:")
    print(f"   Min:    {df_result['slope'].min():.2f}°")
    print(f"   Median: {df_result['slope'].median():.2f}°")
    print(f"   Mean:   {df_result['slope'].mean():.2f}°")
    print(f"   Max:    {df_result['slope'].max():.2f}°")

    return df_result


def classify_risk(df):
    """
    Classify landslide risk based on slope thresholds.

    Risk Categories:
    - Safe: < 10° (gentle slope)
    - Moderate: 10-20° (caution needed)
    - High Risk: > 20° (landslide danger)

    Parameters:
    -----------
    df : pd.DataFrame
        Must contain 'slope' column

    Returns:
    --------
    pd.DataFrame
        DataFrame with added 'risk_level' column
    """
    print("\n" + "="*70)
    print("  STEP 5: CLASSIFYING LANDSLIDE RISK")
    print("="*70)

    # Define risk thresholds
    def get_risk_level(slope):
        if slope < 10:
            return 'Safe'
        elif slope <= 20:
            return 'Moderate'
        else:
            return 'High Risk'

    df['risk_level'] = df['slope'].apply(get_risk_level)

    print(" Risk classification complete")
    print("\n Risk Thresholds:")
    print("   🟢 Safe:      Slope < 10°")
    print("   🟡 Moderate:  Slope 10° - 20°")
    print("   🔴 High Risk: Slope > 20°")

    # Summary statistics
    risk_counts = df['risk_level'].value_counts()
    print(f"\n📊 Overall Risk Distribution:")
    for risk, count in risk_counts.items():
        pct = count / len(df) * 100
        print(f"   {risk}: {count:,} ({pct:.1f}%)")

    return df


In [31]:

# STEP 6: GENERATE VERDICT TABLE

def print_verdict_table(df):
    """
    Generate cross-tabulation of Settlement Type vs Risk Level.
    Highlights the key question: Informal buildings in High Risk zones.

    Parameters:
    -----------
    df : pd.DataFrame
        Must contain 'settlement_type' and 'risk_level'
    """
    print("\n" + "="*70)
    print("🔍 THE VERDICT: SETTLEMENT TYPE vs LANDSLIDE RISK")
    print("="*70)

    # Create cross-tabulation
    crosstab = pd.crosstab(
        df['settlement_type'],
        df['risk_level'],
        margins=True,  # Add totals
        margins_name='Total'
    )

    # Reorder columns for readability
    col_order = ['Safe', 'Moderate', 'High Risk', 'Total']
    crosstab = crosstab[[col for col in col_order if col in crosstab.columns]]

    print("\n📊 Cross-Tabulation (Counts):")
    print(crosstab)

    # Calculate percentages
    crosstab_pct = pd.crosstab(
        df['settlement_type'],
        df['risk_level'],
        normalize='index'  # Percentage within each settlement type
    ) * 100

    crosstab_pct = crosstab_pct[[col for col in ['Safe', 'Moderate', 'High Risk']
                                  if col in crosstab_pct.columns]]

    print("\n📊 Cross-Tabulation (Percentages within Settlement Type):")
    print(crosstab_pct.round(1))

    # THE KEY ANSWER
    print("\n" + "="*70)
    print("🎯 KEY FINDING: INFORMAL SETTLEMENTS IN HIGH RISK ZONES")
    print("="*70)

    informal_high_risk = df[
        (df['settlement_type'] == 'Informal') &
        (df['risk_level'] == 'High Risk')
    ]

    total_informal = len(df[df['settlement_type'] == 'Informal'])
    informal_high_risk_count = len(informal_high_risk)
    informal_high_risk_pct = (informal_high_risk_count / total_informal * 100) if total_informal > 0 else 0

    print(f"\n CRITICAL: {informal_high_risk_count:,} Informal buildings in High Risk zones")
    print(f"   ({informal_high_risk_pct:.1f}% of all Informal settlements)")

    # Additional context
    if informal_high_risk_pct > 15:
        print(f"\n  WARNING: More than 15% of informal settlements are on")
        print(f"   steep slopes (>20°). Urgent landslide mitigation needed.")

    # Compare with Planned/Commercial
    planned_high_risk = len(df[
        (df['settlement_type'] == 'Planned') &
        (df['risk_level'] == 'High Risk')
    ])
    commercial_high_risk = len(df[
        (df['settlement_type'] == 'Commercial') &
        (df['risk_level'] == 'High Risk')
    ])

    print(f"\n Comparison:")
    print(f"   Informal:   {informal_high_risk_count:,} in High Risk")
    print(f"   Planned:    {planned_high_risk:,} in High Risk")
    print(f"   Commercial: {commercial_high_risk:,} in High Risk")

    return crosstab, crosstab_pct



In [32]:

# =============================================================================
# MAIN EXECUTION PIPELINE
# =============================================================================

def analyze_slope_risk(df):
    """
    Complete pipeline for slope-based landslide risk analysis.

    Parameters:
    -----------
    df : pd.DataFrame
        Must contain columns: ['lat', 'lon', 'settlement_type', 'building_area', 'density_50m']

    Returns:
    --------
    pd.DataFrame
        Enhanced dataframe with 'slope' and 'risk_level' columns
    """

    print("\n")
    print("="*70)
    print("🏔️  KIGALI LANDSLIDE RISK ANALYSIS")
    print("="*70)
    print(f"📅 Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"📊 Input Buildings: {len(df):,}")
    print("="*70)

    # Step 1: Get slope layer
    slope_image = get_slope_layer()

    # Step 2: Convert DataFrame to Earth Engine FeatureCollection
    fc = df_to_ee_features(df)

    # Step 3: Sample slope at building locations
    sampled_fc = sample_slope_at_buildings(fc, slope_image)

    # Step 4: Convert back to DataFrame
    df_with_slope = ee_to_dataframe(sampled_fc)

    # Step 5: Classify risk
    df_with_slope = classify_risk(df_with_slope)

    # Step 6: Generate verdict table
    crosstab, crosstab_pct = print_verdict_table(df_with_slope)

    print("\n" + "="*70)
    print("✅ ANALYSIS COMPLETE")
    print("="*70)
    print(f"New columns added: ['slope', 'risk_level']")
    print(f"\nTo use: df_risk = analyze_slope_risk(df)")
    print("="*70)

    return df_with_slope


In [33]:
"""
Urban DNA Sequencer - Slope-Based Vulnerability Analysis (BATCHED VERSION)
===========================================================================
Uses Google Earth Engine with batch processing to analyze landslide risk
for large datasets (>5000 buildings). Handles EE's element limit gracefully.

Author: Claude
Date: 2026-02-10
Version: 2.0 (Batch Processing)
"""

import ee
import pandas as pd
import numpy as np
from datetime import datetime
import time

# =============================================================================
# STEP 1: LOAD NASA SRTM ELEVATION DATA & CALCULATE SLOPE
# =============================================================================

def get_slope_layer(roi_bounds=None):
    """
    Load SRTM elevation data and calculate slope in degrees.

    Parameters:
    -----------
    roi_bounds : list, optional
        Bounding box [min_lon, min_lat, max_lon, max_lat]

    Returns:
    --------
    ee.Image
        Slope image in degrees
    """
    print("="*70)
    print("🏔️  STEP 1: LOADING NASA SRTM ELEVATION DATA")
    print("="*70)

    # Load SRTM Digital Elevation Model (30m resolution)
    srtm = ee.Image('USGS/SRTMGL1_003')
    elevation = srtm.select('elevation')

    print("✅ SRTM DEM loaded (30m resolution)")

    # Calculate slope in degrees
    slope = ee.Terrain.slope(elevation)

    print("✅ Slope calculated (in degrees)")
    print(f"   Range: 0° (flat) to ~90° (vertical cliff)")

    return slope


# =============================================================================
# STEP 2: BATCH PROCESSING FUNCTION (HANDLES >5000 ELEMENTS)
# =============================================================================

def process_slope_in_batches(df, slope_image, chunk_size=4000):
    """
    Process large DataFrame in batches to avoid EE's 5000 element limit.

    Parameters:
    -----------
    df : pd.DataFrame
        Must contain 'lat', 'lon', 'settlement_type', 'building_area', 'density_50m'
    slope_image : ee.Image
        Slope raster in degrees
    chunk_size : int
        Number of buildings per batch (default: 4000, safe under 5000 limit)

    Returns:
    --------
    pd.DataFrame
        DataFrame with added 'slope' column
    """

    print("\n" + "="*70)
    print("🔄 STEP 2: BATCH PROCESSING (Chunking Strategy)")
    print("="*70)

    total_buildings = len(df)
    num_chunks = int(np.ceil(total_buildings / chunk_size))

    print(f"📊 Total Buildings: {total_buildings:,}")
    print(f"📦 Chunk Size: {chunk_size:,} buildings/batch")
    print(f"🔢 Number of Batches: {num_chunks}")
    print(f"⏱️  Estimated Time: ~{num_chunks * 0.5:.1f} minutes")
    print("="*70)

    # Master list to collect results
    all_results = []
    failed_batches = []

    # Process in chunks
    for i in range(num_chunks):
        batch_num = i + 1
        start_idx = i * chunk_size
        end_idx = min((i + 1) * chunk_size, total_buildings)

        # Get chunk
        chunk_df = df.iloc[start_idx:end_idx].copy()
        chunk_size_actual = len(chunk_df)

        print(f"\n📦 Batch {batch_num}/{num_chunks}")
        print(f"   Rows: {start_idx:,} to {end_idx:,} ({chunk_size_actual:,} buildings)")

        try:
            # Convert chunk to FeatureCollection
            print(f"   🔄 Converting to FeatureCollection...")
            features = []

            for idx, row in chunk_df.iterrows():
                point = ee.Geometry.Point([row['lon'], row['lat']])
                feature = ee.Feature(point, {
                    'original_index': int(idx),
                    'settlement_type': str(row['settlement_type']),
                    'building_area': float(row['building_area']),
                    'density_50m': int(row['density_50m'])
                })
                features.append(feature)

            fc = ee.FeatureCollection(features)

            # Sample slope values
            print(f"   🌍 Sampling slope from SRTM...")
            sampled = slope_image.reduceRegions(
                collection=fc,
                reducer=ee.Reducer.first(),
                scale=30,
                tileScale=4
            )

            # Download results with retry logic
            print(f"   ⬇️  Downloading data from Earth Engine...")

            max_retries = 3
            for attempt in range(max_retries):
                try:
                    result = sampled.getInfo()
                    break
                except Exception as e:
                    if attempt < max_retries - 1:
                        wait_time = 2 ** attempt  # Exponential backoff: 1s, 2s, 4s
                        print(f"   ⚠️  Retry {attempt + 1}/{max_retries} after {wait_time}s...")
                        time.sleep(wait_time)
                    else:
                        raise e

            # Extract data
            for feature in result['features']:
                props = feature['properties']
                all_results.append({
                    'original_index': props.get('original_index'),
                    'slope': props.get('first', None)  # 'first' is the reducer output
                })

            print(f"   ✅ Batch {batch_num} complete ({len(result['features']):,} records)")

            # Small delay to avoid rate limiting
            if batch_num < num_chunks:
                time.sleep(0.5)

        except Exception as e:
            print(f"   ❌ ERROR in Batch {batch_num}: {str(e)}")
            print(f"   ⚠️  Filling {chunk_size_actual} buildings with slope=0 (SAFE FALLBACK)")

            # Add failed batch with default slope=0
            for idx in chunk_df.index:
                all_results.append({
                    'original_index': int(idx),
                    'slope': 0.0
                })

            failed_batches.append(batch_num)

    # Convert results to DataFrame
    print("\n" + "="*70)
    print("📥 CONSOLIDATING BATCH RESULTS")
    print("="*70)

    results_df = pd.DataFrame(all_results)

    # Sort by original index to maintain order
    results_df = results_df.sort_values('original_index').reset_index(drop=True)

    print(f"✅ Retrieved {len(results_df):,} slope values")

    if failed_batches:
        print(f"⚠️  {len(failed_batches)} batch(es) failed: {failed_batches}")
        print(f"   ({len(failed_batches) * chunk_size:,} buildings defaulted to slope=0)")
    else:
        print(f"🎉 All {num_chunks} batches processed successfully!")

    # Handle missing values
    missing_count = results_df['slope'].isna().sum()
    if missing_count > 0:
        print(f"\n⚠️  {missing_count} buildings have missing slope data")
        print(f"   Filling with median slope value...")
        median_slope = results_df['slope'].median()
        results_df['slope'].fillna(median_slope, inplace=True)

    # Add slope column to original dataframe
    df['slope'] = results_df['slope'].values

    print(f"\n📊 Slope Statistics:")
    print(f"   Min:    {df['slope'].min():.2f}°")
    print(f"   Median: {df['slope'].median():.2f}°")
    print(f"   Mean:   {df['slope'].mean():.2f}°")
    print(f"   Max:    {df['slope'].max():.2f}°")

    return df


# =============================================================================
# STEP 3: RISK CLASSIFICATION
# =============================================================================

def classify_risk(df):
    """
    Classify landslide risk based on slope thresholds.

    Risk Categories:
    - Safe: < 10° (gentle slope)
    - Moderate: 10-20° (caution needed)
    - High Risk: > 20° (landslide danger)

    Parameters:
    -----------
    df : pd.DataFrame
        Must contain 'slope' column

    Returns:
    --------
    pd.DataFrame
        DataFrame with added 'risk_level' column
    """
    print("\n" + "="*70)
    print("⚠️  STEP 3: CLASSIFYING LANDSLIDE RISK")
    print("="*70)

    # Define risk thresholds
    def get_risk_level(slope):
        if slope < 10:
            return 'Safe'
        elif slope <= 20:
            return 'Moderate'
        else:
            return 'High Risk'

    df['risk_level'] = df['slope'].apply(get_risk_level)

    print("✅ Risk classification complete")
    print("\n📋 Risk Thresholds:")
    print("   🟢 Safe:      Slope < 10°")
    print("   🟡 Moderate:  Slope 10° - 20°")
    print("   🔴 High Risk: Slope > 20°")

    # Summary statistics
    risk_counts = df['risk_level'].value_counts()
    print(f"\n📊 Overall Risk Distribution:")
    for risk in ['Safe', 'Moderate', 'High Risk']:
        if risk in risk_counts.index:
            count = risk_counts[risk]
            pct = count / len(df) * 100
            print(f"   {risk}: {count:,} ({pct:.1f}%)")

    return df


# =============================================================================
# STEP 4: GENERATE VERDICT TABLE
# =============================================================================

def print_verdict_table(df):
    """
    Generate cross-tabulation of Settlement Type vs Risk Level.
    Highlights the key question: Informal buildings in High Risk zones.

    Parameters:
    -----------
    df : pd.DataFrame
        Must contain 'settlement_type' and 'risk_level'
    """
    print("\n" + "="*70)
    print("🔍 THE VERDICT: SETTLEMENT TYPE vs LANDSLIDE RISK")
    print("="*70)

    # Create cross-tabulation
    crosstab = pd.crosstab(
        df['settlement_type'],
        df['risk_level'],
        margins=True,  # Add totals
        margins_name='Total'
    )

    # Reorder columns for readability
    col_order = ['Safe', 'Moderate', 'High Risk', 'Total']
    crosstab = crosstab[[col for col in col_order if col in crosstab.columns]]

    print("\n📊 Cross-Tabulation (Counts):")
    print(crosstab)

    # Calculate percentages
    crosstab_pct = pd.crosstab(
        df['settlement_type'],
        df['risk_level'],
        normalize='index'  # Percentage within each settlement type
    ) * 100

    crosstab_pct = crosstab_pct[[col for col in ['Safe', 'Moderate', 'High Risk']
                                  if col in crosstab_pct.columns]]

    print("\n📊 Cross-Tabulation (Percentages within Settlement Type):")
    print(crosstab_pct.round(1))

    # THE KEY ANSWER
    print("\n" + "="*70)
    print("🎯 KEY FINDING: INFORMAL SETTLEMENTS IN HIGH RISK ZONES")
    print("="*70)

    informal_high_risk = df[
        (df['settlement_type'] == 'Informal') &
        (df['risk_level'] == 'High Risk')
    ]

    total_informal = len(df[df['settlement_type'] == 'Informal'])
    informal_high_risk_count = len(informal_high_risk)
    informal_high_risk_pct = (informal_high_risk_count / total_informal * 100) if total_informal > 0 else 0

    print(f"\n🔴 CRITICAL: {informal_high_risk_count:,} Informal buildings in High Risk zones")
    print(f"   ({informal_high_risk_pct:.1f}% of all Informal settlements)")

    # Mean slope for high-risk informal buildings
    if len(informal_high_risk) > 0:
        mean_slope_high_risk = informal_high_risk['slope'].mean()
        max_slope_high_risk = informal_high_risk['slope'].max()
        print(f"\n   Average slope: {mean_slope_high_risk:.1f}°")
        print(f"   Steepest slope: {max_slope_high_risk:.1f}°")

    # Additional context
    if informal_high_risk_pct > 15:
        print(f"\n⚠️  WARNING: More than 15% of informal settlements are on")
        print(f"   steep slopes (>20°). Urgent landslide mitigation needed.")
    elif informal_high_risk_pct > 10:
        print(f"\n⚠️  CAUTION: {informal_high_risk_pct:.1f}% of informal settlements")
        print(f"   are in landslide-prone areas. Monitoring recommended.")
    else:
        print(f"\n✅ RELATIVELY SAFE: Only {informal_high_risk_pct:.1f}% of informal")
        print(f"   settlements are on steep slopes.")

    # Compare with Planned/Commercial
    planned_high_risk = len(df[
        (df['settlement_type'] == 'Planned') &
        (df['risk_level'] == 'High Risk')
    ])
    commercial_high_risk = len(df[
        (df['settlement_type'] == 'Commercial') &
        (df['risk_level'] == 'High Risk')
    ])

    total_planned = len(df[df['settlement_type'] == 'Planned'])
    total_commercial = len(df[df['settlement_type'] == 'Commercial'])

    print(f"\n📊 Comparative Risk Analysis:")
    print(f"   Informal:   {informal_high_risk_count:,}/{total_informal:,} in High Risk ({informal_high_risk_pct:.1f}%)")
    print(f"   Planned:    {planned_high_risk:,}/{total_planned:,} in High Risk ({planned_high_risk/total_planned*100 if total_planned > 0 else 0:.1f}%)")
    print(f"   Commercial: {commercial_high_risk:,}/{total_commercial:,} in High Risk ({commercial_high_risk/total_commercial*100 if total_commercial > 0 else 0:.1f}%)")

    return crosstab, crosstab_pct


# =============================================================================
# MAIN EXECUTION PIPELINE (BATCHED VERSION)
# =============================================================================

def analyze_slope_risk(df, chunk_size=4000):
    """
    Complete pipeline for slope-based landslide risk analysis with batch processing.

    Parameters:
    -----------
    df : pd.DataFrame
        Must contain columns: ['lat', 'lon', 'settlement_type', 'building_area', 'density_50m']
    chunk_size : int
        Number of buildings per batch (default: 4000, safe under EE's 5000 limit)

    Returns:
    --------
    pd.DataFrame
        Enhanced dataframe with 'slope' and 'risk_level' columns
    """

    print("\n")
    print("="*70)
    print("🏔️  KIGALI LANDSLIDE RISK ANALYSIS (BATCHED VERSION)")
    print("="*70)
    print(f"📅 Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"📊 Input Buildings: {len(df):,}")
    print(f"🔧 Batch Processing: Enabled (chunk_size={chunk_size:,})")
    print("="*70)

    # Verify required columns
    required_cols = ['lat', 'lon', 'settlement_type', 'building_area', 'density_50m']
    missing_cols = [col for col in required_cols if col not in df.columns]

    if missing_cols:
        raise ValueError(f"Missing required columns: {missing_cols}")

    # Step 1: Get slope layer
    slope_image = get_slope_layer()

    # Step 2: Process in batches
    df_with_slope = process_slope_in_batches(df, slope_image, chunk_size=chunk_size)

    # Step 3: Classify risk
    df_with_slope = classify_risk(df_with_slope)

    # Step 4: Generate verdict table
    crosstab, crosstab_pct = print_verdict_table(df_with_slope)

    print("\n" + "="*70)
    print("✅ ANALYSIS COMPLETE")
    print("="*70)
    print(f"New columns added: ['slope', 'risk_level']")
    print(f"\nTo use: df_risk = analyze_slope_risk(df)")
    print("="*70)

    return df_with_slope


In [34]:
# 1. CONNECT TO EARTH ENGINE


# 2. RUN THE BATCHED SCAN
# Watch the progress bars! It will say "Batch 1/6", "Batch 2/6", etc.
df = analyze_slope_risk(df, chunk_size=4000)

# 3. EXPORT THE DANGER LIST
# This saves the specific list of families who need relocation.
export_high_risk_buildings(df, 'High_Risk_Red_Zone.csv')



🏔️  KIGALI LANDSLIDE RISK ANALYSIS (BATCHED VERSION)
📅 Analysis Date: 2026-02-17 15:50:13
📊 Input Buildings: 21,035
🔧 Batch Processing: Enabled (chunk_size=4,000)
🏔️  STEP 1: LOADING NASA SRTM ELEVATION DATA
✅ SRTM DEM loaded (30m resolution)
✅ Slope calculated (in degrees)
   Range: 0° (flat) to ~90° (vertical cliff)

🔄 STEP 2: BATCH PROCESSING (Chunking Strategy)
📊 Total Buildings: 21,035
📦 Chunk Size: 4,000 buildings/batch
🔢 Number of Batches: 6
⏱️  Estimated Time: ~3.0 minutes

📦 Batch 1/6
   Rows: 0 to 4,000 (4,000 buildings)
   🔄 Converting to FeatureCollection...
   🌍 Sampling slope from SRTM...
   ⬇️  Downloading data from Earth Engine...
   ✅ Batch 1 complete (4,000 records)

📦 Batch 2/6
   Rows: 4,000 to 8,000 (4,000 buildings)
   🔄 Converting to FeatureCollection...
   🌍 Sampling slope from SRTM...
   ⬇️  Downloading data from Earth Engine...
   ✅ Batch 2 complete (4,000 records)

📦 Batch 3/6
   Rows: 8,000 to 12,000 (4,000 buildings)
   🔄 Converting to FeatureCollection..

NameError: name 'export_high_risk_buildings' is not defined

# **Final Strategy Map (Action Plan)**

Creates an actionable intervention map for Kigali urban planning:
- RED: High-priority relocation zones (Informal + High Risk)
- TEAL: Upgrading zones (Informal + Safe/Moderate)
- GRAY: Stable zones (Planned/Commercial)

In [35]:
"""
Urban DNA Sequencer - Final Strategy Map (Action Plan)
=======================================================
Creates an actionable intervention map for Kigali urban planning:
- RED: High-priority relocation zones (Informal + High Risk)
- TEAL: Upgrading zones (Informal + Safe/Moderate)
- GRAY: Stable zones (Planned/Commercial)

Author: Claude
Date: 2026-02-10
"""

import folium
from folium import plugins
import pandas as pd

def create_action_plan_map(df, output_file='kigali_action_plan.html'):
    """
    Generate strategic intervention map with prioritized action zones.

    Parameters:
    -----------
    df : pd.DataFrame
        Must contain: ['lat', 'lon', 'settlement_type', 'risk_level', 'slope', 'density_50m']
    output_file : str
        Output HTML filename

    Returns:
    --------
    folium.Map
        The generated strategic map
    """

    print("="*70)
    print("🎯 GENERATING KIGALI STRATEGIC ACTION PLAN MAP")
    print("="*70)

    # =============================================================================
    # STEP 1: CLASSIFY BUILDINGS INTO ACTION ZONES
    # =============================================================================

    print("\n🏘️  Classifying Action Zones...")

    def assign_action_zone(row):
        """Determine action required based on settlement type and risk."""
        if row['settlement_type'] == 'Informal':
            if row['risk_level'] == 'High Risk':
                return 'Relocation Zone'
            else:
                return 'Upgrading Zone'
        else:
            return 'Stable Zone'

    def assign_action_label(row):
        """Get action label for tooltip."""
        if row['settlement_type'] == 'Informal':
            if row['risk_level'] == 'High Risk':
                return 'RELOCATE'
            else:
                return 'UPGRADE'
        else:
            return 'Monitor'

    # Add action columns
    df['action_zone'] = df.apply(assign_action_zone, axis=1)
    df['action_required'] = df.apply(assign_action_label, axis=1)

    # Count by zone
    zone_counts = df['action_zone'].value_counts()
    print(f"\n📊 Action Zone Distribution:")
    for zone, count in zone_counts.items():
        pct = count / len(df) * 100
        print(f"   {zone}: {count:,} ({pct:.1f}%)")

    # =============================================================================
    # STEP 2: DEFINE COLOR SCHEME & STYLING
    # =============================================================================

    # Color and style configuration
    zone_config = {
        'Relocation Zone': {
            'color': '#FF0000',      # RED - Urgent action
            'radius': 4,             # Larger for visibility
            'opacity': 0.9,
            'weight': 2
        },
        'Upgrading Zone': {
            'color': '#008080',      # TEAL - Medium priority
            'radius': 2,
            'opacity': 0.7,
            'weight': 1
        },
        'Stable Zone': {
            'color': '#D3D3D3',      # GRAY - Low priority
            'radius': 1,
            'opacity': 0.3,          # Very transparent
            'weight': 0.5
        }
    }

    # =============================================================================
    # STEP 3: CREATE BASE MAP
    # =============================================================================

    print("\n🗺️  Creating base map...")

    center_lat = df['lat'].mean()
    center_lon = df['lon'].mean()

    # Create map with canvas rendering for performance
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=13,
        tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr='Esri WorldImagery',
        prefer_canvas=True  # Performance optimization
    )

    # =============================================================================
    # STEP 4: ADD MARKERS IN LAYERS (BOTTOM TO TOP)
    # =============================================================================

    print("\n📌 Adding markers by priority...")

    # Create feature groups for each zone
    feature_groups = {}
    for zone in ['Stable Zone', 'Upgrading Zone', 'Relocation Zone']:
        if zone in zone_counts.index:
            count = zone_counts[zone]
            feature_groups[zone] = folium.FeatureGroup(
                name=f"{zone} ({count:,})",
                show=True
            )

    # Add markers in order: Stable → Upgrading → Relocation
    # This ensures RED dots appear on top
    layer_order = ['Stable Zone', 'Upgrading Zone', 'Relocation Zone']

    for zone in layer_order:
        if zone not in feature_groups:
            continue

        zone_df = df[df['action_zone'] == zone]
        config = zone_config[zone]

        for idx, row in zone_df.iterrows():
            # Create tooltip with action details
            tooltip_text = f"""
            <div style="font-family: Arial; font-size: 12px;">
                <b>Action Required:</b> {row['action_required']}<br>
                <b>Slope:</b> {row['slope']:.1f}°
            </div>
            """

            # Create circle marker
            folium.CircleMarker(
                location=[row['lat'], row['lon']],
                radius=config['radius'],
                color=config['color'],
                fill=True,
                fillColor=config['color'],
                fillOpacity=config['opacity'],
                weight=config['weight'],
                tooltip=tooltip_text
            ).add_to(feature_groups[zone])

    # Add feature groups to map in order
    for zone in layer_order:
        if zone in feature_groups:
            feature_groups[zone].add_to(m)

    # =============================================================================
    # STEP 5: ADD LAYER CONTROL
    # =============================================================================

    # Add layer control
    folium.LayerControl(
        collapsed=False,
        position='topright'
    ).add_to(m)

    # Add fullscreen button
    plugins.Fullscreen(
        position='topleft',
        title='Fullscreen',
        title_cancel='Exit Fullscreen'
    ).add_to(m)

    # =============================================================================
    # STEP 6: SAVE MAP
    # =============================================================================

    m.save(output_file)

    print("\n" + "="*70)
    print("✅ MAP GENERATION COMPLETE")
    print("="*70)
    print(f"📄 Output File: {output_file}")
    print("="*70)

    return m


# =============================================================================
# USAGE EXAMPLE (FOR COLAB)
# =============================================================================

if __name__ == "__main__":
    print("\n💡 USAGE IN COLAB:")
    print("="*70)
    print("# Generate the action plan map")
    print("map_object = create_action_plan_map(df)")
    print()
    print("# Download the map")
    print("from google.colab import files")
    print("files.download('kigali_action_plan.html')")
    print("="*70)


💡 USAGE IN COLAB:
# Generate the action plan map
map_object = create_action_plan_map(df)

# Download the map
from google.colab import files
files.download('kigali_action_plan.html')


In [36]:
# Generate the map
map_object = create_action_plan_map(df)

# Download it
from google.colab import files
files.download('kigali_action_plan.html')

🎯 GENERATING KIGALI STRATEGIC ACTION PLAN MAP

🏘️  Classifying Action Zones...

📊 Action Zone Distribution:
   Upgrading Zone: 11,489 (54.6%)
   Stable Zone: 9,005 (42.8%)
   Relocation Zone: 541 (2.6%)

🗺️  Creating base map...

📌 Adding markers by priority...

✅ MAP GENERATION COMPLETE
📄 Output File: kigali_action_plan.html


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [37]:
!pip install keplergl --quiet
!pip install joblib --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.4/18.4 MB 82.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 123.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 101.5 MB/s eta 0:00:00


In [38]:
"""
Urban DNA Sequencer - Kepler.gl 3D Visualization & Model Deployment
====================================================================
High-end geospatial visualization using Kepler.gl with model serialization
for production deployment.

Author: Claude
Date: 2026-02-10
"""

# =============================================================================
# STEP 0: INSTALLATION (Run this cell first in Colab)
# =============================================================================



# =============================================================================
# IMPORTS
# =============================================================================

import pandas as pd
import numpy as np
from keplergl import KeplerGl
import joblib
import json
from datetime import datetime

# =============================================================================
# STEP 1: DATA PREPARATION FOR KEPLER.GL
# =============================================================================

def prepare_data_for_kepler(df):
    """
    Clean and prepare DataFrame for Kepler.gl visualization.
    Ensures all categorical columns are strings and data types are correct.

    Parameters:
    -----------
    df : pd.DataFrame
        Raw dataframe with urban DNA features

    Returns:
    --------
    pd.DataFrame
        Clean dataframe ready for Kepler.gl
    """

    print("="*70)
    print("🧹 STEP 1: DATA PREPARATION FOR KEPLER.GL")
    print("="*70)

    # Create a copy to avoid modifying original
    df_clean = df.copy()

    # Convert categorical columns to strings
    categorical_cols = ['settlement_type', 'risk_level', 'action_zone', 'action_required']

    for col in categorical_cols:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].astype(str)
            print(f"✅ Converted '{col}' to string")

    # Ensure numeric columns are proper numeric types
    numeric_cols = ['lat', 'lon', 'building_area', 'shape_index',
                    'nearest_dist', 'density_50m', 'slope']

    for col in numeric_cols:
        if col in df_clean.columns:
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

    # Handle any NaN values
    nan_count = df_clean.isnull().sum().sum()
    if nan_count > 0:
        print(f"\n⚠️  Found {nan_count} NaN values, filling with defaults...")
        df_clean = df_clean.fillna({
            'slope': 0,
            'density_50m': 0,
            'building_area': 0,
            'shape_index': 1,
            'nearest_dist': 0
        })

    print(f"\n📊 Final Dataset:")
    print(f"   Rows: {len(df_clean):,}")
    print(f"   Columns: {len(df_clean.columns)}")
    print(f"   Data Types:")
    for col in df_clean.columns:
        print(f"      - {col}: {df_clean[col].dtype}")

    return df_clean


# =============================================================================
# STEP 2: CREATE KEPLER.GL VISUALIZATION
# =============================================================================

def create_kepler_map(df, output_file='kigali_kepler_viz.html'):
    """
    Create a high-end 3D geospatial visualization using Kepler.gl.

    Parameters:
    -----------
    df : pd.DataFrame
        Clean dataframe with urban DNA features
    output_file : str
        Output HTML filename

    Returns:
    --------
    KeplerGl
        The Kepler.gl map object
    """

    print("\n" + "="*70)
    print("🗺️  STEP 2: CREATING KEPLER.GL VISUALIZATION")
    print("="*70)

    # Initialize Kepler.gl map
    print("\n🚀 Initializing Kepler.gl...")

    # Custom configuration for better visualization
    config = {
        'version': 'v1',
        'config': {
            'mapState': {
                'latitude': df['lat'].mean(),
                'longitude': df['lon'].mean(),
                'zoom': 12,
                'pitch': 45,  # 3D tilt
                'bearing': 0
            },
            'mapStyle': {
                'styleType': 'satellite'
            }
        }
    }

    # Create map with configuration
    map_kepler = KeplerGl(height=800, config=config)

    # Add data
    print(f"📊 Adding dataset: 'Kigali_Urban_DNA' ({len(df):,} buildings)")
    map_kepler.add_data(data=df, name='Kigali_Urban_DNA')

    print("✅ Data successfully added to Kepler.gl")

    # Save to HTML
    print(f"\n💾 Saving map to '{output_file}'...")
    map_kepler.save_to_html(file_name=output_file)

    # Get file size
    import os
    file_size_mb = os.path.getsize(output_file) / (1024 * 1024)

    print(f"✅ Map saved successfully!")
    print(f"   File: {output_file}")
    print(f"   Size: {file_size_mb:.2f} MB")

    return map_kepler


# =============================================================================
# STEP 3: SAVE ML MODEL FOR DEPLOYMENT
# =============================================================================

def save_ml_model(kmeans_model, scaler, output_file='urban_dna_brain.pkl'):
    """
    Serialize the trained K-Means model and StandardScaler for deployment.

    Parameters:
    -----------
    kmeans_model : sklearn.cluster.KMeans
        Trained K-Means clustering model
    scaler : sklearn.preprocessing.StandardScaler
        Fitted StandardScaler used for feature normalization
    output_file : str
        Output pickle filename

    Returns:
    --------
    dict
        Model metadata for verification
    """

    print("\n" + "="*70)
    print("🧠 STEP 3: SAVING ML MODEL FOR DEPLOYMENT")
    print("="*70)

    # Create model package with metadata
    model_package = {
        'kmeans_model': kmeans_model,
        'scaler': scaler,
        'metadata': {
            'model_type': 'KMeans',
            'n_clusters': kmeans_model.n_clusters,
            'features': ['building_area', 'shape_index', 'nearest_dist', 'density_50m'],
            'training_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'version': '1.0',
            'inertia': kmeans_model.inertia_,
            'n_iter': kmeans_model.n_iter_
        }
    }

    # Save using joblib (more efficient for sklearn models)
    print(f"\n💾 Serializing model to '{output_file}'...")
    joblib.dump(model_package, output_file, compress=3)

    # Get file size
    import os
    file_size_kb = os.path.getsize(output_file) / 1024

    print(f"✅ Model saved successfully!")
    print(f"   File: {output_file}")
    print(f"   Size: {file_size_kb:.2f} KB")

    # Print model metadata
    print(f"\n📋 Model Metadata:")
    print(f"   Model Type: {model_package['metadata']['model_type']}")
    print(f"   Clusters: {model_package['metadata']['n_clusters']}")
    print(f"   Features: {model_package['metadata']['features']}")
    print(f"   Training Date: {model_package['metadata']['training_date']}")
    print(f"   Inertia: {model_package['metadata']['inertia']:.2f}")
    print(f"   Iterations: {model_package['metadata']['n_iter']}")

    return model_package['metadata']


# =============================================================================
# STEP 4: LOAD ML MODEL (FOR FUTURE INFERENCE)
# =============================================================================

def load_ml_model(model_file='urban_dna_brain.pkl'):
    """
    Load the saved ML model for inference on new data.

    Parameters:
    -----------
    model_file : str
        Path to saved model file

    Returns:
    --------
    tuple
        (kmeans_model, scaler, metadata)
    """

    print("\n" + "="*70)
    print("📥 LOADING SAVED ML MODEL")
    print("="*70)

    # Load model package
    print(f"🔄 Loading model from '{model_file}'...")
    model_package = joblib.load(model_file)

    kmeans_model = model_package['kmeans_model']
    scaler = model_package['scaler']
    metadata = model_package['metadata']

    print(f"✅ Model loaded successfully!")
    print(f"\n📋 Model Info:")
    print(f"   Type: {metadata['model_type']}")
    print(f"   Clusters: {metadata['n_clusters']}")
    print(f"   Trained: {metadata['training_date']}")

    return kmeans_model, scaler, metadata


# =============================================================================
# STEP 5: PREDICT ON NEW DATA (DEPLOYMENT EXAMPLE)
# =============================================================================

def predict_settlement_type(new_buildings_df, model_file='urban_dna_brain.pkl'):
    """
    Use the saved model to classify new buildings.

    Parameters:
    -----------
    new_buildings_df : pd.DataFrame
        New buildings with features: ['building_area', 'shape_index', 'nearest_dist', 'density_50m']
    model_file : str
        Path to saved model

    Returns:
    --------
    pd.DataFrame
        DataFrame with added 'settlement_type' predictions
    """

    print("\n" + "="*70)
    print("🔮 PREDICTING SETTLEMENT TYPES FOR NEW BUILDINGS")
    print("="*70)

    # Load model
    kmeans_model, scaler, metadata = load_ml_model(model_file)

    # Extract features
    features = metadata['features']
    X_new = new_buildings_df[features].values

    print(f"\n📊 Input Data: {len(new_buildings_df):,} buildings")

    # Scale features
    X_scaled = scaler.transform(X_new)

    # Predict clusters
    cluster_labels = kmeans_model.predict(X_scaled)

    # Map clusters to settlement types (using same logic as training)
    # You'll need to adjust this based on your actual cluster mapping
    label_map = {0: 'Informal', 1: 'Planned', 2: 'Commercial'}
    new_buildings_df['settlement_type'] = [label_map[c] for c in cluster_labels]

    print(f"✅ Predictions complete!")
    print(f"\n📊 Predicted Distribution:")
    print(new_buildings_df['settlement_type'].value_counts())

    return new_buildings_df


# =============================================================================
# MAIN EXECUTION PIPELINE
# =============================================================================

def main(df, kmeans_model, scaler):
    """
    Complete pipeline: Data prep → Kepler.gl viz → Model serialization.

    Parameters:
    -----------
    df : pd.DataFrame
        Full dataset with all features
    kmeans_model : sklearn.cluster.KMeans
        Trained K-Means model
    scaler : sklearn.preprocessing.StandardScaler
        Fitted scaler

    Returns:
    --------
    tuple
        (kepler_map, model_metadata)
    """

    print("\n")
    print("="*70)
    print("🚀 URBAN DNA SEQUENCER - FINAL DEPLOYMENT PIPELINE")
    print("="*70)
    print(f"📅 Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"📊 Total Buildings: {len(df):,}")
    print("="*70)

    # Step 1: Prepare data
    df_clean = prepare_data_for_kepler(df)

    # Step 2: Create Kepler.gl visualization
    kepler_map = create_kepler_map(df_clean, output_file='kigali_kepler_viz.html')

    # Step 3: Save ML model
    model_metadata = save_ml_model(kmeans_model, scaler, output_file='urban_dna_brain.pkl')

    print("\n" + "="*70)
    print("✅ DEPLOYMENT PIPELINE COMPLETE")
    print("="*70)
    print("\n📦 Generated Files:")
    print("   1. kigali_kepler_viz.html  (3D Interactive Map)")
    print("   2. urban_dna_brain.pkl     (ML Model Package)")

    print("\n🎯 Next Steps:")
    print("   1. Display Kepler map in Colab:")
    print("      kepler_map")
    print("\n   2. Download files:")
    print("      from google.colab import files")
    print("      files.download('kigali_kepler_viz.html')")
    print("      files.download('urban_dna_brain.pkl')")

    print("\n   3. Deploy model on new data:")
    print("      predictions = predict_settlement_type(new_df)")

    print("="*70)

    return kepler_map, model_metadata


# =============================================================================
# USAGE EXAMPLE (FOR COLAB)
# =============================================================================

if __name__ == "__main__":
    print("\n💡 COMPLETE USAGE IN COLAB:")
    print("="*70)
    print("""
# ========================================
# STEP 0: INSTALL DEPENDENCIES
# ========================================
!pip install keplergl --quiet
!pip install joblib --quiet

# ========================================
# STEP 1: IMPORT AND RUN PIPELINE
# ========================================
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Assuming you have:
# - df: DataFrame with all features
# - kmeans: Trained KMeans model
# - scaler: Fitted StandardScaler

# Run the complete pipeline
kepler_map, metadata = main(df, kmeans, scaler)

# ========================================
# STEP 2: DISPLAY KEPLER MAP IN COLAB
# ========================================
kepler_map

# ========================================
# STEP 3: DOWNLOAD FILES
# ========================================
from google.colab import files

# Download Kepler.gl visualization
files.download('kigali_kepler_viz.html')

# Download trained model
files.download('urban_dna_brain.pkl')

# ========================================
# STEP 4: (OPTIONAL) TEST MODEL LOADING
# ========================================
kmeans_loaded, scaler_loaded, metadata = load_ml_model('urban_dna_brain.pkl')
print(f"Model loaded successfully! Clusters: {metadata['n_clusters']}")

# ========================================
# STEP 5: (FUTURE) USE MODEL ON NEW DATA
# ========================================
# When you get new building data:
# predictions = predict_settlement_type(new_buildings_df, 'urban_dna_brain.pkl')
    """)
    print("="*70)


# =============================================================================
# KEPLER.GL TIPS & TRICKS
# =============================================================================

def print_kepler_tips():
    """
    Print helpful tips for using Kepler.gl visualization.
    """
    print("\n" + "="*70)
    print("💡 KEPLER.GL VISUALIZATION TIPS")
    print("="*70)
    print("""
🎨 LAYERS PANEL (Left Side):
   - Click "+ Add Layer" to visualize different attributes
   - Try: Point layer colored by 'settlement_type'
   - Try: Hexbin layer to see density clusters
   - Try: 3D Hexagon layer (height = density_50m)

🎛️ FILTERS PANEL:
   - Filter by 'risk_level' to see only High Risk buildings
   - Filter by 'settlement_type' to focus on Informal settlements
   - Combine filters: Informal + High Risk = Relocation Zone

🗺️ BASE MAP:
   - Switch between Satellite, Dark, Light themes
   - Satellite view best for ground-truthing against real imagery

📊 RECOMMENDED VISUALIZATIONS:
   1. Point Map: Color by 'action_zone' (Red/Teal/Gray)
   2. Heatmap: Show density of high-risk areas
   3. 3D Hexagons: Height = slope, Color = settlement_type
   4. Arc Layer: Connect buildings to nearest road (if you have road data)

⚡ PERFORMANCE:
   - For 21k points, Kepler.gl handles it smoothly
   - Use aggregation layers (Hexbin) for cleaner views
   - Toggle layers on/off to reduce rendering load

💾 EXPORT:
   - Click camera icon (top-right) to export as PNG
   - Share the HTML file with stakeholders (fully interactive)
    """)
    print("="*70)


# Print tips when module is loaded
if __name__ == "__main__":
    print_kepler_tips()


💡 COMPLETE USAGE IN COLAB:

# ========================================
# STEP 0: INSTALL DEPENDENCIES
# ========================================
!pip install keplergl --quiet
!pip install joblib --quiet

# ========================================
# STEP 1: IMPORT AND RUN PIPELINE
# ========================================
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Assuming you have:
# - df: DataFrame with all features
# - kmeans: Trained KMeans model
# - scaler: Fitted StandardScaler

# Run the complete pipeline
kepler_map, metadata = main(df, kmeans, scaler)

# ========================================
# STEP 2: DISPLAY KEPLER MAP IN COLAB
# ========================================
kepler_map

# ========================================
# STEP 3: DOWNLOAD FILES
# ========================================
from google.colab import files

# Download Kepler.gl visualization
files.download('kigali_kepler_viz.html')

# Download t

In [41]:
# Step 1: Install dependencies (already executed, but keeping for completeness)
!pip install keplergl joblib --quiet

# Re-extract the features and re-train KMeans to get the kmeans model and scaler object
# These were generated internally by the previous main(df) call, but not returned.
# The 'df' DataFrame already contains the 'density_50m' column from the prior run.
features = ['building_area', 'shape_index', 'nearest_dist', 'density_50m']
# The train_kmeans_with_scaling function was defined in cell mIcyx1J8h16r
cluster_labels, scaler, kmeans = train_kmeans_with_scaling(df, features, n_clusters=3)

# Step 2: Run complete pipeline using the retrieved kmeans and scaler
kepler_map, metadata = main(df, kmeans, scaler)

# Step 3: Display map in Colab
kepler_map

# Step 4: Download files
from google.colab import files
files.download('kigali_kepler_viz.html')
files.download('urban_dna_brain.pkl')


 Step 2: Training K-Means (k=3) with StandardScaler...
 Feature ranges BEFORE scaling:
   - building_area: [3.02, 5721.05]
   - shape_index: [15.12, 95.73]
   - nearest_dist: [1.46, 133.15]
   - density_50m: [0.00, 57.00]
   ✅ Clustering complete. Cluster sizes: [ 7970 12030  1035]


🚀 URBAN DNA SEQUENCER - FINAL DEPLOYMENT PIPELINE
📅 Date: 2026-02-17 15:57:43
📊 Total Buildings: 21,035
🧹 STEP 1: DATA PREPARATION FOR KEPLER.GL
✅ Converted 'settlement_type' to string
✅ Converted 'risk_level' to string
✅ Converted 'action_zone' to string
✅ Converted 'action_required' to string

📊 Final Dataset:
   Rows: 21,035
   Columns: 12
   Data Types:
      - building_area: float64
      - shape_index: float64
      - lat: float64
      - lon: float64
      - nearest_dist: float64
      - density_50m: int64
      - cluster_id: int32
      - settlement_type: object
      - slope: float64
      - risk_level: object
      - action_zone: object
      - action_required: object

🗺️  STEP 2: CREATING KEPLE

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>